In [1]:
# Dependencies & Installs
import pandas as pd
import streamlit as st
import pickle

In [3]:
# Load data from S3 into a Pandas DataFrame (data needs to be scaled and cleaned already)
# s3_file_path = "s3://esophageal-cancer-biochem-data/Joined_df_cleaned.csv"
# df = pd.read_csv(s3_file_path)

local_file_path = "Data_Cleaned/User_Samples/users.csv"
df = pd.read_csv(local_file_path)
df.head()

,Patient Group,O95445_AAL,P00450_AAL,P00734_AAL,P00736_AAL,P00738_AAL,P00747_AAL,P00748_AAL,P00751_AAL,P01008_AAL,...,P02748_LSPIYNLVPVK_Ratio_NPL,P04114_SPAFTDLHLR_Ratio_NPL,P06396_AVEVLPK_Ratio_NPL,P0C0L5_GSFEFPVGDAVSK_Ratio_NPL,P10643_LTPLYELVK_Ratio_NPL,Protein (μg/μL),Age at Collection,BMI (kg/m2),Gender_F,Gender_M
0,BE-LGD,235835.96960,446414.537900,50712.825780,1.264925e+05,3.367665e+06,1.460257e+06,185795.700300,4.849568e+05,2.695428e+05,...,1.626764,4.462419,1.133528,0.649006,1.089551,76.593205,55.0,27.70,0,1
1,EAC,253204.99200,548152.870400,53040.291110,1.170803e+05,9.780856e+06,1.276418e+06,102165.973400,4.233527e+05,2.068396e+05,...,0.682581,4.710647,1.187228,0.409018,0.965052,67.596795,65.0,32.60,0,1
2,BE-LGD,223918.83730,668157.986100,49551.798430,1.143228e+05,2.616339e+07,1.378266e+06,135028.685100,4.678381e+05,1.393544e+05,...,1.183182,3.761115,1.377637,0.627450,1.275043,74.833366,55.0,30.20,1,0
3,NSE,317408.86981,624661.826663,166048.185854,5.879864e+07,3.151861e+07,2.006994e+06,66677.865179,8.918962e+06,3.406724e+06,...,1.858580,11.655492,4.471990,0.821500,2.668353,72.455779,54.0,30.04,1,0
4,BE-HGD,251668.16160,345571.798200,67323.520520,1.379693e+05,2.220131e+07,1.482350e+06,86503.256360,4.053318e+05,2.758485e+05,...,0.859171,6.339829,1.221672,0.580200,2.279625,80.726013,63.0,33.50,0,1


In [9]:
# Load the trained model
with open('Models/Model_Saved/model_rf_LogisticRegression.pkl', 'rb') as f:
    model_rf = pickle.load(f)

In [14]:
# Define Streamlit app
def app():
    
    # Set app title
    st.title('Esophageal Cancer Risk Assessment app')

    # Add some text
    st.write("Welcome to our Esophageal Cancer Risk Assessment app! This app utilizes advanced machine learning algorithms to estimate your risk of developing esophageal cancer based on pre-screening and blood sample data. Esophageal cancer is a life-threatening disease affecting millions of people worldwide, and early diagnosis is crucial for improving survival rates. Traditional diagnostic methods, such as endoscopy, can be invasive and expensive. Our app aims to provide a faster, more affordable, and less invasive alternative by leveraging machine learning techniques like logistic regression, decision trees, and support vector machines. Using a dataset of biochemical data from patients with varying esophageal conditions, our models have been trained and evaluated to deliver accurate predictions. Get started by inputting your data to assess your esophageal cancer risk.")
    st.image("Images/ai-generated-image-dalle.png", use_column_width=True)

    # Get user input
    age = st.number_input("Enter your age", value=30, min_value=18, max_value=100)
    sex = st.selectbox("Select your sex", ["male", "female"])
    bmi = st.number_input("Enter your BMI", value=25, min_value=0, max_value=50)
    diagnosed = st.selectbox("Have you been diagnosed?", ["No", "Barrett esophagus - no dysplasia", "Barrett esophagus - low dysplasia", "Barrett esophagus - high dysplasia", "Esophageal cancer"])

    patient_group = ""
    if diagnosed == "No":
        patient_group = "NSE"
    elif diagnosed == "Barrett esophagus - no dysplasia":
        patient_group = "BE"
    elif diagnosed == "Barrett esophagus - low dysplasia":
        patient_group = "BE-LGD"
    elif diagnosed == "Barrett esophagus - high dysplasia":
        patient_group = "BE-HGD"
    elif diagnosed == "Esophageal cancer":
        patient_group = "EAC"

    # Create a DataFrame with the user input
    user_input = pd.DataFrame({
        "Age at Collection": [age],
        "sex": [sex],
        "BMI (kg/m2)": [bmi]
    })

    # If the user clicks the "Generate Blood Test Results" button, fetch a sample row from the dataset
    if st.button("Generate Blood Test Results"):
        sample_row = df.sample(n=1)
        blood_results_df = sample_row.drop(columns=["Patient Group", "Age at Collection", "BMI (kg/m2)", "Gender_F", "Gender_M"])
        user_input = pd.concat([user_input, blood_results_df.reset_index(drop=True)], axis=1)

        # Make a prediction using the model
        prediction = model_rf.predict(user_input)

        # Display the prediction
        if prediction[0] == 1:
            st.write("You have a high risk of developing esophageal cancer.")
        else:
            st.write("You have a low risk of developing esophageal cancer.")
    else:
        st.write("Click the button to generate blood test results and make a prediction.")

# Run the Streamlit app
if __name__ == '__main__':
    app()